In [5]:
# Libraries
from fpdf import FPDF
import pandas as pd
import yaml
import re


In [6]:
# Title    
title_report = "Prices Report"


In [7]:
class report_pages:
    def __init__(self, header_img_path, chart_img_paths, output_file):
        self.pdf = FPDF()
        self.header_img_path = header_img_path
        self.chart_img_paths = chart_img_paths
        self.output_file = output_file
        
        # Preload Montserrat fonts
        try:
            self.pdf.add_font('Montserrat-Bold', 'B', './assets/Montserrat/static/Montserrat-Bold.ttf', uni=True)
            self.pdf.add_font('Montserrat-Regular', '', './assets/Montserrat/static/Montserrat-Regular.ttf', uni=True)
            self.pdf.add_font('Montserrat-Italic', 'I', './assets/Montserrat/static/Montserrat-Italic.ttf', uni=True)
        except RuntimeError as e:
            print(f"Error adding font: {e}")
            raise
        
    def add_table(self):
        """Add CSV table to the report with specific formatting."""

        self.pdf.ln(15)
        
        # Load the CSV file into a DataFrame
        csv_file_path = '../data/processed/metrics_output.csv'  # Replace with your actual file path
        df = pd.read_csv(csv_file_path)
        
        # Ensure Montserrat fonts are preloaded in the class `__init__` method
        self.pdf.set_font('Montserrat-Bold', 'B', 10)  # Header font
        self.pdf.set_text_color(0, 0, 0)  # RGB for black

        # Margins and column widths
        left_margin = 10  # Small left margin
        first_col_width = 50  # Width for the first column
        num_columns = len(df.columns)
        other_col_width = (self.pdf.w - left_margin * 2 - first_col_width) / (num_columns - 1)

        # Move to starting position with left margin
        self.pdf.set_x(left_margin)

        # Write the header row
        for i, column in enumerate(df.columns):
            col_width = first_col_width if i == 0 else other_col_width
            self.pdf.cell(col_width, 8, str(column), border='B', align="C")
        self.pdf.ln()  # Move to the next row

        # Set data font
        self.pdf.set_font('Montserrat-Regular', '', 9)  # Data font

        # Detect percentage columns
        percentage_columns = [col for col in df.columns if "%" in col.lower() or "percentage" in col.lower()]

        # Write the data rows
        for idx, row in df.iterrows():
            self.pdf.set_x(left_margin)  # Reset position to the left margin
            for i, (column, item) in enumerate(row.items()):
                col_width = first_col_width if i == 0 else other_col_width

                # Format numbers
                if isinstance(item, (int, float)):
                    if column in percentage_columns:
                        item_formatted = f"{item:,.2f}"  # Retain 2 decimals for percentage columns
                    else:
                        item_formatted = f"{item:,.2f}"  # Format with thousand separators, no decimals
                    
                    # Set color for negative numbers
                    if item < 0:
                        self.pdf.set_text_color(255, 0, 0)  # Red for negative numbers
                    else:
                        self.pdf.set_text_color(0, 0, 0)  # Black for positive numbers
                else:
                    item_formatted = str(item)
                    self.pdf.set_text_color(0, 0, 0)  # Default to black for non-numeric items

                self.pdf.cell(col_width, 6, item_formatted, align="C", border=0)  # Write the cell
            self.pdf.ln()  # Move to the next row

        # Reset text color to black
        self.pdf.set_text_color(0, 0, 0)

        # Add a line above the "Source" note
        self.pdf.ln(2)  # Space above the line
        self.pdf.set_draw_color(0, 0, 0)  # Set the line color to black
        self.pdf.line(left_margin, self.pdf.get_y(), self.pdf.w - left_margin, self.pdf.get_y())  # Draw the line
        self.pdf.ln(2)  # Space below the line

        # Add a note below the table in italics
        note_text = "Source: Yahoo Finance"
        self.pdf.set_font('Montserrat-Italic', 'I', 8)
        self.pdf.set_xy(left_margin, self.pdf.get_y())  # Set the row position
        self.pdf.multi_cell(0, 4, note_text, align="R")  # Write the note text with automatic line breaks
        
    def add_singular_chart(self, num_pos):
        """Add the chart images to the PDF"""
        chart_image_path = self.chart_img_paths[num_pos]
        y_position = self.pdf.get_y() + 0  # Update the y_position after adding the image
        page_width = self.pdf.w
        image_width = 200
        left_margin = (page_width - image_width) / 2  # Center the image
        self.pdf.image(chart_image_path, x=left_margin, y=y_position, w=image_width)
        self.pdf.ln(90)

    def add_header(self):
        """Add the header image and title to the PDF"""
        self.pdf.add_page()
        self.pdf.image(self.header_img_path, x=0, y=0, w=211)
        self.pdf.set_text_color(255, 255, 255)  # RGB for black
        self.pdf.set_xy(1, 14)
        self.pdf.add_font('Montserrat-Bold', 'B', './assets/Montserrat/static/Montserrat-Bold.ttf', uni=True)
        self.pdf.set_font('Montserrat-Bold', 'B', 24)
        self.pdf.multi_cell(0, 10, f"{title_report}", 0, 'L')
        
    def generate_report(self, title_report):
        """Generate and save the report as a PDF"""
        # Page 1
        self.add_header()
        self.add_table()
        self.add_singular_chart(0)
        self.pdf.output(self.output_file)

In [8]:
# Usage example:
header_img_path = './assets/header.png'
chart_img_paths = ["../report/images/treemap.png"

]

# Get today's date in the desired format
from datetime import datetime
today_date = datetime.now().strftime("%Y %m %d")

# Generate the output file path with the formatted date
output_file = f'../report/{title_report} - {today_date}.pdf'

report = report_pages(header_img_path, chart_img_paths, output_file)
report.generate_report(title_report)